In [ ]:
import requests

from bs4 import BeautifulSoup

url = "https://www.vivino.com/es/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660TS1WS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2hYUZSanqpUXR8faGloYqBUBaRMDUwBHtxqD"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
}

response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")
# Leemos el contenido de la web
response.content
print(soup)

In [ ]:
# Mostramos las cabeceras de la HTTP
response.headers

In [ ]:
soup.find("title")

In [ ]:
# Seleccionamos los p elementos
soup.select("p")

In [ ]:
# Mostramos como quedó el parseo para poder leerlo
print(soup.prettify()[:2000])

In [ ]:
# Exploramos el contenido de la pagina       ## CONTAR A MARTA ESTO SACA LOS ELEMENTOS DE LA PAGINA
wines = soup.find_all("div", class_="wineCard__wineCard--2dj2T wineCard__large--1tkVl")
print(len(wines))

In [ ]:
# Exploramos el contenido de la pagina  ## CONTAR A MARTA ESTO SOLO SACA TEXTO, SI QUEREMOS REALMENTE TODA LA INFO DEL INTERIOR TENEMOS QUE IR AL PADRE
title = soup.find_all("div", class_="wineInfoVintage__truncate--3QAtw")
print(title)

In [ ]:
titles = [w.get_text(strip=True) for w in title]
titles

In [ ]:
rating = soup.find_all("div", class_="vivinoRating__averageValue--3p6Wp")
print(rating)

In [ ]:
ratings = [r.get_text(strip=True) for r in rating]
print(ratings)

In [ ]:
price = soup.find_all("div", class_="addToCartButton__price--qJdh4")
print(price)

In [ ]:
prices = [p.get_text(strip=True) for p in price]
print(prices)

In [ ]:
# Exploramos el contenido del primer elemento de la pagina
print(title[0].prettify())

In [ ]:
wine_block = title[0].find_parent()
print(wine_block.prettify())

In [ ]:
# Exploramos los vinos de la primera pagina
wine_divs = soup.find_all("div", class_="wineInfoVintage__wineInfoVintage--bXr7s") #CONTAR A MARTA ESTO SACA AL PADRE ES MAS SUPERIOR QUE EL OTRO ES EL CONTEDOR
print(wine_divs)

In [ ]:
# Exploramos el primer elemento de la pagina
first_wine = wine_divs[0]
print(first_wine)

In [ ]:
first_wine = soup.find("div", class_="wineInfoVintage__wineInfoVintage--bXr7s")

In [ ]:
print(type(first_wine))
print(first_wine)

In [ ]:
container = first_wine.find_parent()
print(container.prettify())

In [ ]:
# VALE A PARTIR DE AQUI PARA WEB SCRAPING
# Esto lee cada pagina y extrae el html
import requests
import csv
import time

# ==============================
# CONFIGURACIÓN
# ==============================

base_url = "https://www.vivino.com/api/explore/explore"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Parámetros base para buscar vinos tintos de España
params = {
    "country_code": "ES",
    "country_codes[]": "es",
    "wine_type_ids[]": "1",   # 1 = vinos tintos
    "order_by": "ratings_average",
    "order": "desc",
    "page": 1
}

all_wines = []

# ==============================
# BUCLE DE EXTRACCIÓN
# ==============================

for page in range(1, 500):   # Cambia 90 por más páginas si quieres
    print(f"\n🔎 Explorando página {page}...")
    params["page"] = page

    res = requests.get(base_url, params=params, headers=headers)
    
    # Si falla la petición, salimos
    if res.status_code != 200:
        print(f"⚠️ Error HTTP {res.status_code}, deteniendo scraping.")
        break

    data = res.json()

    matches = data.get("explore_vintage", {}).get("matches", [])
    print(f"   → {len(matches)} vinos encontrados en la página")

    # Si ya no hay resultados, paramos
    if not matches:
        print("🛑 No hay más vinos. Fin del scraping.")
        break

    for m in matches:
        vintage = m.get("vintage", {})
        wine = vintage.get("wine", {})

        if not wine:
            continue

        region_name = wine.get("region", {}).get("name", "").lower()

        # ==============================
        # 📌 FILTRO: SOLO RIBERA DEL DUERO
        # ==============================
        if "ribera" not in region_name and "duero" not in region_name:
            continue

        # Construcción correcta de la URL
        vintage_id = vintage.get("id")
        price_id = vintage.get("price", {}).get("id")

        url = f"https://www.vivino.com/es{wine.get('seo_name','')}/w/{wine.get('id','')}"

        if vintage_id:
            url += f"?year={vintage_id}"
        if price_id:
            url += f"&price_id={price_id}"

        # Datos del vino
        info = {
            "Nombre": wine.get("name"),
            "Año": vintage.get("year"),
            "Bodega": wine.get("winery", {}).get("name"),
            "Región": wine.get("region", {}).get("name"),
            "Uvas": ", ".join(g["name"] for g in wine.get("grapes", []) if wine.get("grapes")) ,
            "Estilo": wine.get("style", {}).get("name"),
            "Rating promedio": vintage.get("statistics", {}).get("ratings_average"),
            "Número de valoraciones": vintage.get("statistics", {}).get("ratings_count"),
            "Precio (EUR)": vintage.get("price", {}).get("amount"),
            "URL": url
        }

        all_wines.append(info)
        print(f"🍷 Añadido: {info['Nombre']} → {info['Región']}")

    time.sleep(1)  # evitar ser bloqueado

# ==============================
# GUARDADO EN CSV
# ==============================

if all_wines:
    filename = "vivino_ribera_del_duero.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=all_wines[0].keys())
        writer.writeheader()
        writer.writerows(all_wines)

    print(f"\n💾 Archivo guardado: {filename} ({len(all_wines)} vinos encontrados)")
else:
    print("⚠️ No se encontraron vinos de Ribera del Duero.")


In [ ]:
# Import necessary libraries
import pandas as pd
import time
import random
from pathlib import Path

In [ ]:
# ¡Ejecuta esta celda primero para preparar Selenium!

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

print("✓ Selenium listo para usarse")

In [ ]:
df = pd.read_csv('vivino_ribera_del_duero.csv')
df['URL'] = df['URL'].apply(lambda x: x.split('?')[0])

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path
import random

# -------------------------------
# Crear driver de Chrome
# -------------------------------
def _create_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--start-maximized")
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument(
        "--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"
    )
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=chrome_options)

# -------------------------------
# Generar nombre de archivo HTML por URL base
# -------------------------------
def get_filename_from_url(url, html_dir="wine_html_1"):
    Path(html_dir).mkdir(exist_ok=True)
    wine_id = url.rstrip('/').split('/')[-1].split('?')[0]  # solo ID del vino
    filename = f"{wine_id}.html"
    return Path(html_dir) / filename

# -------------------------------
# Función principal para descargar HTML
# -------------------------------
def get_wine_details(url, html_dir="wine_html_1", wait_seconds=5, headless=True):
    driver = None
    try:
        print(f"\nCargando URL: {url}")
        driver = _create_driver(headless=headless)
        driver.set_page_load_timeout(60)

        driver.get(url)

        WebDriverWait(driver, 30).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
        time.sleep(wait_seconds)

        html_content = driver.page_source

        filepath = get_filename_from_url(url, html_dir)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(html_content)

        print(f"✓ Guardado en: {filepath}")
        return True, filepath

    except (TimeoutException, WebDriverException) as e:
        return False, f"Error del navegador: {e}"
    except Exception as e:
        return False, f"Error inesperado: {e}"
    finally:
        if driver is not None:
            driver.quit()

# -------------------------------
# Función para recorrer CSV y descargar HTML
# -------------------------------
def scrape_wine_csv(csv_file, html_dir="wine_html", headless=True, filter_region="Ribera del Duero"):
    df = pd.read_csv(csv_file)

    # Filtrar solo vinos de Ribera del Duero si se desea
    if filter_region:
        df = df[df['Región'] == filter_region].copy()
        print(f"Se van a procesar {len(df)} vinos de la región '{filter_region}'.")

    # Crear columna con URL base sin parámetros
    df['url_base'] = df['URL'].apply(lambda x: x.split('?')[0])

    # Identificar URLs únicas para descargar HTML
    unique_urls = df['url_base'].unique()
    print(f"Se van a descargar {len(unique_urls)} páginas HTML únicas.")

    html_results = []
    for i, url in enumerate(unique_urls, 1):
        print(f"\n{'='*60}\nHTML {i} de {len(unique_urls)}\nURL: {url}\n{'='*60}")
        success, filepath = get_wine_details(url, html_dir=html_dir, headless=headless)
        html_results.append({'url_base': url, 'success': success, 'filepath': filepath})
        if i < len(unique_urls):
            time.sleep(random.randint(3, 6))  # espera aleatoria entre descargas

    # Guardar resultados de HTML descargados
    html_results_df = pd.DataFrame(html_results)
    html_results_df.to_csv("html_download_results.csv", index=False)

    # Guardar CSV final combinando HTML descargado con todas las añadas
    final_df = df.copy()
    final_df = final_df.merge(html_results_df[['url_base','filepath']], on='url_base', how='left')
    final_df.to_csv("vivino_wine_final.csv", index=False)

    print("\nSCRAPING COMPLETO! CSV final guardado como 'vivino_wine_final.csv'.")
    return final_df

# -------------------------------
# EJEMPLO DE USO
# -------------------------------
if __name__ == "__main__":
    # Cambia el nombre del CSV por tu archivo real
    final_df = scrape_wine_csv("vivino_ribera_del_duero.csv", html_dir="wine_html_1", headless=True)


In [ ]:
# LEE LOS HTML Y LOS EXPORTA A EXCEL
# -*- coding: utf-8 -*-
"""
Extrae información de vinos desde los HTML de Vivino guardados en la carpeta wine_html
y genera el fichero Excel: vivino_wines_extracted.xlsx

Columnas:
wine_id, nombre, seo, precio, rating, año, n_ratings, bodega,
uvas, region, wine_style, alcohol, alergenos, descripcion, source_file
"""

import os
import re
import json
import glob
import pandas as pd
from bs4 import BeautifulSoup


# ---------- Helpers para extraer JSON principal ----------

def extract_wpi_json(html: str):
    """
    Extrae el objeto JSON que corresponde al valor de "winePageInformation"
    buscando el texto "winePageInformation" y haciendo matching de llaves { }.
    Devuelve un dict (winePageInformation) o lanza excepción si falla.
    """
    marker = '"winePageInformation"'
    idx = html.find(marker)
    if idx == -1:
        raise ValueError("No se encontró el marcador winePageInformation")

    colon = html.find(":", idx + len(marker))
    if colon == -1:
        raise ValueError("No se encontró ':' después de winePageInformation")

    start = html.find("{", colon)
    if start == -1:
        raise ValueError("No se encontró '{' después de ':'")

    depth = 0
    in_str = False
    escape = False
    end = None

    for i in range(start, len(html)):
        ch = html[i]
        if in_str:
            if escape:
                escape = False
            elif ch == '\\':
                escape = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                depth += 1
            elif ch == '}':
                depth -= 1
                if depth == 0:
                    end = i + 1
                    break

    if end is None:
        raise ValueError("No se cerraron las llaves de winePageInformation")

    json_str = html[start:end]
    return json.loads(json_str)


# ---------- Helpers HTML (bodega / alcohol / alérgenos) ----------

def extract_bodega_from_html(html: str):
    """
    Busca la fila de Wine Facts cuyo label es 'Bodega' y lee el texto de la celda <td>.
    Ejemplo: Dominio de Pingus, Altanza, etc.
    """
    soup = BeautifulSoup(html, "html.parser")

    for tr in soup.select('tr[data-testid="wineFactRow"]'):
        header_span = tr.find(
            "span",
            string=lambda t: t and "Bodega" in t
        )
        if header_span:
            td = tr.find('td')
            if td:
                txt = td.get_text(strip=True)
                if txt:
                    return txt
    return None


def extract_alcohol_allergens_from_html(html: str):
    """
    Fallback HTML:
    - Busca fila 'Contenido de alcohol' y extrae algo tipo '14%' -> 14.0
    - Busca fila 'Alérgenos' y mapea a sulfitos / lácteos / huevo si se indica.
    """
    soup = BeautifulSoup(html, "html.parser")

    alcohol_val = None
    allergens_set = set()

    for tr in soup.select('tr[data-testid="wineFactRow"]'):
        # span con el label de la fila (Estilo de vino, Contenido de alcohol, Alérgenos, etc.)
        label_span = tr.find(
            "span",
            class_=lambda c: c and "wineFacts__headerLabel" in c
        )
        if not label_span:
            continue

        label = label_span.get_text(strip=True)

        # Celda con el valor
        td = tr.find("td", class_=lambda c: c and "wineFacts__fact" in c)
        if not td:
            td = tr.find("td")
        if not td:
            continue

        value_text = td.get_text(strip=True)

        # Contenido de alcohol
        if any(x in label for x in ["Contenido de alcohol", "Alcohol content"]):
            m = re.search(r"([\d.,]+)", value_text)
            if m:
                num_str = m.group(1).replace(",", ".")
                try:
                    alcohol_val = float(num_str)
                except Exception:
                    pass

        # Alérgenos
        if any(x in label for x in ["Alérgenos", "Allergens"]):
            vlow = value_text.lower()
            if "sulfit" in vlow:
                allergens_set.add("sulfitos")
            if "leche" in vlow or "milk" in vlow:
                allergens_set.add("lácteos")
            if "huevo" in vlow or "egg" in vlow:
                allergens_set.add("huevo")

    allergens_txt = ", ".join(sorted(allergens_set)) if allergens_set else None
    return {"alcohol_html": alcohol_val, "alergenos_html": allergens_txt}


# ---------- Helpers JSON-LD (fallback precio/rating/año) ----------

def extract_from_ld_json(html: str):
    """
    Fallback usando <script type="application/ld+json"> (schema.org Product).
    Extrae:
      - price_ld_eur: precio en EUR (prefiriendo ofertas para ES si existen)
      - rating_ld: rating medio
      - n_ratings_ld: número de valoraciones
      - year_ld_from_name: año detectado en el nombre del producto (si aparece 19xx/20xx)
    """
    soup = BeautifulSoup(html, "html.parser")

    price_eur = None
    price_currency = None
    rating = None
    n_ratings = None
    year_from_name = None

    for script in soup.find_all("script", type="application/ld+json"):
        if not script.string:
            continue
        txt = script.string.strip()

        try:
            data = json.loads(txt)
        except Exception:
            last = txt.rfind('}')
            if last != -1:
                try:
                    data = json.loads(txt[:last + 1])
                except Exception:
                    continue
            else:
                continue

        objs = data if isinstance(data, list) else [data]

        for obj in objs:
            if not isinstance(obj, dict):
                continue
            if obj.get("@type") != "Product":
                continue

            # Año desde el nombre del producto
            name = obj.get("name") or ""
            m = re.search(r"(19|20)\d{2}", name)
            if m and not year_from_name:
                year_from_name = int(m.group())

            # Rating / nº valoraciones
            agg = obj.get("aggregateRating") or {}
            if agg:
                if rating is None and agg.get("ratingValue"):
                    try:
                        rating = float(agg["ratingValue"])
                    except Exception:
                        pass
                if n_ratings is None and agg.get("ratingCount"):
                    try:
                        n_ratings = int(agg["ratingCount"])
                    except Exception:
                        pass

            # Ofertas
            offers = obj.get("offers") or []
            offers = offers if isinstance(offers, list) else [offers]

            best_es = None
            best_eur = None

            for off in offers:
                if not isinstance(off, dict):
                    continue
                cur = off.get("priceCurrency")

                if off.get("@type") == "AggregateOffer":
                    price_str = off.get("lowPrice") or off.get("highPrice")
                else:
                    price_str = off.get("price")

                if not price_str:
                    continue

                try:
                    p = float(str(price_str).replace(",", "."))
                except Exception:
                    continue

                reg = off.get("eligibleRegion") or {}
                reg_name = reg.get("name")

                if cur == "EUR" and reg_name == "ES":
                    best_es = (p, cur)
                    break
                if cur == "EUR" and best_eur is None:
                    best_eur = (p, cur)

            chosen = best_es or best_eur
            if chosen and price_eur is None:
                price_eur, price_currency = chosen

    return {
        "price_ld_eur": price_eur,
        "currency_ld": price_currency,
        "rating_ld": rating,
        "n_ratings_ld": n_ratings,
        "year_ld_from_name": year_from_name,
    }


# ---------- Fallback directo a prices_and_availability cuando NO hay winePageInformation ----------

def extract_price_year_from_availability(html: str):
    """
    Busca un bloque JSON de '"prices_and_availability" : { ... }' en el HTML
    (fuera de winePageInformation) y devuelve precio y año si existen.
    """
    marker = '"prices_and_availability"'
    idx = html.find(marker)
    if idx == -1:
        return {"price_from_ppa": None, "year_from_ppa": None}

    colon = html.find(":", idx + len(marker))
    if colon == -1:
        return {"price_from_ppa": None, "year_from_ppa": None}

    start = html.find("{", colon)
    if start == -1:
        return {"price_from_ppa": None, "year_from_ppa": None}

    depth = 0
    in_str = False
    escape = False
    end = None

    for i in range(start, len(html)):
        ch = html[i]
        if in_str:
            if escape:
                escape = False
            elif ch == '\\':
                escape = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == '{':
                depth += 1
            elif ch == '}':
                depth -= 1
                if depth == 0:
                    end = i + 1
                    break

    if end is None:
        return {"price_from_ppa": None, "year_from_ppa": None}

    try:
        json_str = html[start:end]
        data = json.loads(json_str)
    except Exception:
        return {"price_from_ppa": None, "year_from_ppa": None}

    availability = data.get("availability") or {}
    vintage = availability.get("vintage") or {}
    median = availability.get("median") or {}
    price_entry = availability.get("price") or {}

    year = vintage.get("year")

    amount = None
    if price_entry.get("amount") is not None:
        amount = price_entry.get("amount")
    elif median.get("amount") is not None:
        amount = median.get("amount")

    price_value = None
    if amount is not None:
        try:
            price_value = float(amount)
        except Exception:
            price_value = None

    return {"price_from_ppa": price_value, "year_from_ppa": year}


# ---------- Extracción principal de cada HTML ----------

def extract_wine_from_html(html: str, source_file: str):
    """
    Devuelve un dict con todos los campos pedidos para un HTML concreto.
    """

    wine_id = None
    nombre = None
    seo = None
    precio = None
    rating = None
    year = None
    n_ratings = None
    bodega = None
    uvas = None
    region = None
    wine_style = None
    alcohol = None
    alergenos = None
    descripcion = None

    # 1) JSON principal winePageInformation
    try:
        wpi = extract_wpi_json(html)
    except Exception as e:
        print(f"[ERROR] {source_file}: no se pudo extraer winePageInformation ({e})")
        # Fallback: JSON-LD + prices_and_availability global + HTML facts
        ld = extract_from_ld_json(html)
        pa = extract_price_year_from_availability(html)
        html_facts = extract_alcohol_allergens_from_html(html)

        precio = ld.get("price_ld_eur") or pa.get("price_from_ppa")
        rating = ld.get("rating_ld")
        year = ld.get("year_ld_from_name") or pa.get("year_from_ppa")
        n_ratings = ld.get("n_ratings_ld")

        bodega = extract_bodega_from_html(html)
        alcohol = html_facts.get("alcohol_html")
        alergenos = html_facts.get("alergenos_html")

        # Último recurso: buscar año en el nombre si algún día lo tenemos en este path
        if year is None and nombre:
            m = re.search(r"(19|20)\d{2}", nombre)
            if m:
                year = int(m.group())

        return {
            "wine_id": wine_id,
            "nombre": nombre,
            "seo": seo,
            "precio": precio,
            "rating": rating,
            "año": year,
            "n_ratings": n_ratings,
            "bodega": bodega,
            "uvas": uvas,
            "region": region,
            "wine_style": wine_style,
            "alcohol": alcohol,
            "alergenos": alergenos,
            "descripcion": descripcion,
            "source_file": source_file,
        }

    wine = wpi.get("wine") or {}
    vintage = wpi.get("vintage") or {}
    region_obj = wine.get("region") or {}
    winery_obj = wine.get("winery") or {}
    wine_style_obj = wine.get("wine_style") or {}

    # ID
    wine_id = vintage.get("id") or wine.get("id")

    # SEO
    seo = wine.get("seo_name") or vintage.get("seo_name")

    # Nombre completo: bodega + nombre vino si hace falta
    base_name = wine.get("name") or vintage.get("name")
    winery_name = winery_obj.get("name")
    full_name = base_name
    if base_name and winery_name:
        if not base_name.lower().startswith(winery_name.lower()):
            full_name = f"{winery_name} {base_name}"
    nombre = full_name

    # Rating y nº valoraciones
    stats_v = vintage.get("statistics") or {}
    stats_w = wine.get("statistics") or {}
    rating = stats_v.get("ratings_average") or stats_w.get("ratings_average")
    n_ratings = stats_v.get("ratings_count") or stats_w.get("ratings_count")

    # Año principal
    if vintage.get("year"):
        year = vintage["year"]
    elif wpi.get("vintage_year"):
        year = wpi["vintage_year"]

    # Precios y año desde prices_and_availabilities (dentro de wpi)
    ppa = (
        wpi.get("prices_and_availabilities")
        or wpi.get("prices_and_availability")
        or []
    )
    if isinstance(ppa, dict):
        ppa = [ppa]

    if ppa:
        chosen_price = None
        chosen_curr = None
        year_from_ppa = None

        for entry in ppa:
            if not isinstance(entry, dict):
                continue

            avail = entry.get("availability") or {}
            vint = avail.get("vintage") or {}
            if not year and vint.get("year"):
                year_from_ppa = vint["year"]

            median = avail.get("median") or entry.get("median") or {}
            amount = median.get("amount")
            curr = (entry.get("market") or {}).get("currency", {}).get("code")

            if amount is None:
                continue

            if chosen_price is None:
                chosen_price, chosen_curr = amount, curr

            if curr == "EUR":
                chosen_price, chosen_curr = amount, curr
                break

        if chosen_price is not None and precio is None:
            try:
                precio = float(chosen_price)
            except Exception:
                pass

        if not year and year_from_ppa:
            year = year_from_ppa

    # Uvas
    grapes = wpi.get("grapes") or wine.get("grapes") or []
    if isinstance(grapes, list):
        uvas = ", ".join(
            sorted(
                {
                    g.get("name")
                    for g in grapes
                    if isinstance(g, dict) and g.get("name")
                }
            )
        ) or None

    # Región
    region = region_obj.get("name")

    # Bodega (JSON primero, si no HTML)
    bodega = winery_name or None
    if not bodega:
        bodega = extract_bodega_from_html(html)

    # Estilo de vino
    wine_style = wine_style_obj.get("name")

    # Alcohol y alérgenos desde JSON
    wine_facts_raw = wpi.get("wine_facts_raw") or wine.get("wine_facts_raw")
    if wine_facts_raw and isinstance(wine_facts_raw, dict):
        if wine_facts_raw.get("alcohol"):
            alcohol = wine_facts_raw["alcohol"]

        flags = []
        if wine_facts_raw.get("contains_sulfites_allergens"):
            flags.append("sulfitos")
        if wine_facts_raw.get("contains_milk_allergens"):
            flags.append("lácteos")
        if wine_facts_raw.get("contains_egg_allergens"):
            flags.append("huevo")
        alergenos = ", ".join(flags) if flags else None

    # Fallback alcohol / alérgenos desde HTML si todavía están vacíos
    html_facts = extract_alcohol_allergens_from_html(html)
    if alcohol is None and html_facts.get("alcohol_html") is not None:
        alcohol = html_facts["alcohol_html"]
    if alergenos is None and html_facts.get("alergenos_html") is not None:
        alergenos = html_facts["alergenos_html"]

    # Descripción
    description = wine.get("description") or vintage.get("description")
    descripcion = description

    # Fallback final desde JSON-LD si faltan precio/rating/año
    ld = extract_from_ld_json(html)

    if precio is None and ld.get("price_ld_eur") is not None:
        precio = ld["price_ld_eur"]

    if rating is None and ld.get("rating_ld") is not None:
        rating = ld["rating_ld"]

    if n_ratings is None and ld.get("n_ratings_ld") is not None:
        n_ratings = ld["n_ratings_ld"]

    if year is None and ld.get("year_ld_from_name") is not None:
        year = ld["year_ld_from_name"]

    # Último recurso: buscar año en el nombre completo
    if year is None and nombre:
        m = re.search(r"(19|20)\d{2}", nombre)
        if m:
            year = int(m.group())

    return {
        "wine_id": wine_id,
        "nombre": nombre,
        "seo": seo,
        "precio": precio,
        "rating": rating,
        "año": year,
        "n_ratings": n_ratings,
        "bodega": bodega,
        "uvas": uvas,
        "region": region,
        "wine_style": wine_style,
        "alcohol": alcohol,
        "alergenos": alergenos,
        "descripcion": descripcion,
        "source_file": source_file,
    }


# ---------- Bucle principal sobre wine_html ----------

def main():
    folder = "wine_html_1"
    pattern = os.path.join(folder, "*.html")
    files = sorted(glob.glob(pattern))

    print(f"Encontrados {len(files)} archivos HTML en {folder}")
    rows = []

    for i, path in enumerate(files, 1):
        with open(path, encoding="utf-8") as f:
            html = f.read()
        fname = os.path.basename(path)

        row = extract_wine_from_html(html, fname)
        rows.append(row)

        if i % 100 == 0:
            print(f"  -> Procesados {i} archivos...")

    df = pd.DataFrame(rows)
    output_file = "vivino_wines_extracted.xlsx"
    df.to_excel(output_file, index=False)
    print(f"Guardado {output_file} con {len(df)} filas")


if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

# ---------------------------
# Cargar archivos
# ---------------------------
vintages = pd.read_csv("vivino_ribera_del_duero.csv")
wines = pd.read_excel("vivino_wines_extracted.xlsx")

# ---------------------------
# Extraer wine_id desde ambas fuentes
# ---------------------------

# Desde la URL del CSV (ej: /w/77137?year=...)
vintages['wine_id'] = (
    vintages['URL']
    .str.extract(r'/w/(\d+)')
    .astype(int)
)

# Desde el nombre del HTML (ej: 77137.html)
wines['wine_id'] = (
    wines['source_file']
    .str.extract(r'(\d+)\.html')
    .astype(int)
)

# ---------------------------
# Unión SIN perder añadas
# ---------------------------
final_df = vintages.merge(
    wines,
    on='wine_id',
    how='left'
)

# ---------------------------
# Comprobaciones rápidas
# ---------------------------
print(f"Filas originales (añadas): {len(vintages)}")
print(f"Filas finales: {len(final_df)}")

missing = final_df['wine_id'].isna().sum()
print(f"Añadas sin vino base asociado: {missing}")

# ---------------------------
# Guardar resultado
# ---------------------------
final_df.to_csv("vivino_ribera_final.csv", index=False)
final_df.to_excel("vivino_ribera_final.xlsx", index=False)

print("✅ Unión completada correctamente.")


In [ ]:
import pandas as pd

df_vivino_completo = pd.read_excel("vivino_ribera_final.xlsx")

df_vivino_completo.head()

In [ ]:
# Filtrar por Bodega y Año
filtro = df_vivino_completo[
    (df_vivino_completo['Bodega'] == "Valtravieso") &
    (df_vivino_completo['Año'] == 2020)
]

# Contar cuántas filas hay
repeticiones = len(filtro)
print(f"El vino 'Vatravieso' del año 2020 se repite {repeticiones} veces.")

# Opcional: ver los detalles de esas filas
filtro


In [ ]:
import requests
import csv
import time
import random

# ==============================
# CONFIGURACIÓN
# ==============================

base_url = "https://www.vivino.com/api/explore/explore"
output_file = "prueba_limpia_4.csv"

# Headers (Si te bloquean, recuerda actualizar Cookie y User-Agent)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Referer": "https://www.vivino.com/explore"
    # "Cookie": "PEGA_AQUI_TU_COOKIE_SI_HACE_FALTA"
}

params = {
    "country_code": "ES",
    "country_codes[]": "es",
    "wine_type_ids[]": "1", # 1 = Tintos
    "order_by": "ratings_average",
    "order": "desc",
    "page": 1,
    "currency_code": "EUR"
}

all_wines = []
seen_ids = set()

# ==============================
# 🧹 FUNCIÓN DE LIMPIEZA
# ==============================
def clean_text(text):
    if text is None: return "N/A"
    text_str = str(text).replace("\n", " ").replace("\r", " ")
    return " ".join(text_str.split())

def yes_no(boolean_value):
    """Convierte True/False en Sí/No para el Excel"""
    return "Sí" if boolean_value else "No"

# ==============================
# BUCLE DE EXTRACCIÓN
# ==============================

for page in range(1, 4):  # ⚠️ Pon aquí cuántas páginas quieres escanear
    print(f"\n🔎 Explorando página {page}...")
    params["page"] = page

    try:
        res = requests.get(base_url, params=params, headers=headers, timeout=10)
    except Exception as e:
        print(f"⚠️ Error de conexión: {e}")
        break
    
    if res.status_code != 200:
        print(f"⚠️ Error HTTP {res.status_code} (Posible bloqueo).")
        break

    data = res.json()
    matches = data.get("explore_vintage", {}).get("matches", [])
    
    if not matches:
        print("🛑 No hay más resultados.")
        break

    count_page = 0

    for m in matches:
        vintage = m.get("vintage", {})
        wine = vintage.get("wine", {})
        
        if not wine: continue

        v_id = vintage.get("id")
        if v_id in seen_ids: continue
        seen_ids.add(v_id)

        # Filtro Ribera del Duero
        region_name = wine.get("region", {}).get("name", "").lower()
        if "ribera" not in region_name and "duero" not in region_name:
            continue

        # --- DATOS BÁSICOS ---
        price_data = m.get("price") or vintage.get("price") or {}
        price_amt = price_data.get("amount")
        price_id = price_data.get("id")
        
        taste = wine.get("taste", {}).get("structure", {})
        style = wine.get("style", {})

        # Listas (Uvas y Maridaje)
        uvas_list = wine.get("grapes", [])
        uvas_str = ", ".join([g.get("name") for g in uvas_list]) if uvas_list else "N/A"

        foods_list = wine.get("foods", [])
        maridaje_str = ", ".join([f.get("name") for f in foods_list]) if foods_list else "N/A"

        # --- URL CORRECTA (Añada + ID Precio) ---
        year_param = vintage.get("year") if vintage.get("year") != "N.V." else v_id
        seo_name = wine.get("seo_name", "wine")
        wine_id = wine.get("id")
        
        url = f"https://www.vivino.com/ES/es/{seo_name}/w/{wine_id}?year={year_param}"
        if price_id:
            url += f"&price_id={price_id}"

        # --- DICCIONARIO DE DATOS ---
        info = {
            "ID": v_id,
            "Nombre Vino": clean_text(wine.get("name")),
            "Año": vintage.get("year"),
            "Precio EUR": price_amt if price_amt else "Agotado",
            "Rating": vintage.get("statistics", {}).get("ratings_average"),
            "Nº Votos": vintage.get("statistics", {}).get("ratings_count"),
            
            # === SECCIÓN: DATOS ACERCA DEL VINO ===
            "Bodega": clean_text(wine.get("winery", {}).get("name")),
            "Región": clean_text(wine.get("region", {}).get("name")),
            "Estilo Regional": clean_text(style.get("regional_name")), # Ej: Ribera del Duero Tinto
            "Uvas": clean_text(uvas_str),
            "Alcohol %": wine.get("alcohol") if wine.get("alcohol") else "N/A",
            # Datos 'ocultos' que a veces salen en esa sección
            "Es Natural": yes_no(wine.get("is_natural")),
            "Es Orgánico": yes_no(wine.get("is_organic")),
            "Es Biodinámico": yes_no(wine.get("is_biodynamic")),
            
            # === RESTO DE DATOS ===
            "Maridaje": clean_text(maridaje_str),
            "Acidez (0-5)": taste.get("acidity"),
            "Taninos (0-5)": taste.get("tannin"),
            "Intensidad (0-5)": taste.get("intensity"),
            "Dulzor (0-5)": taste.get("sweetness"),
            "Descripción": clean_text(style.get("description")),
            "URL": url
        }

        all_wines.append(info)
        count_page += 1

    print(f"🍷 Añadidos: {count_page}")
    time.sleep(random.uniform(1.5, 3.5))

# ==============================
# GUARDADO
# ==============================

if all_wines:
    fieldnames = list(all_wines[0].keys())
    
    with open(output_file, "w", newline="", encoding="utf-8-sig") as f:
        # quoting=csv.QUOTE_ALL asegura que Excel no rompa las columnas
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader()
        writer.writerows(all_wines)

    print(f"\n✅ ÉXITO: Archivo '{output_file}' actualizado con Datos del Vino.")
else:
    print("\n⚠️ No se encontraron datos.")

In [4]:
# -*- coding: utf-8 -*-
"""
Extrae información completa (incluyendo PRECIO coincidente por año)
desde los HTML de Vivino.
"""

import os
import re
import json
import glob
import pandas as pd
from bs4 import BeautifulSoup

# ---------- Helpers para extraer JSON principal ----------

def extract_wpi_json(html: str):
    marker = '"winePageInformation"'
    idx = html.find(marker)
    if idx == -1:
        raise ValueError("No se encontró el marcador winePageInformation")

    colon = html.find(":", idx + len(marker))
    if colon == -1:
        raise ValueError("No se encontró ':' después de winePageInformation")

    start = html.find("{", colon)
    if start == -1:
        raise ValueError("No se encontró '{' después de ':'")

    depth = 0
    in_str = False
    escape = False
    end = None

    for i in range(start, len(html)):
        ch = html[i]
        if in_str:
            if escape: escape = False
            elif ch == '\\': escape = True
            elif ch == '"': in_str = False
        else:
            if ch == '"': in_str = True
            elif ch == '{': depth += 1
            elif ch == '}':
                depth -= 1
                if depth == 0:
                    end = i + 1
                    break

    if end is None:
        raise ValueError("No se cerraron las llaves de winePageInformation")

    json_str = html[start:end]
    return json.loads(json_str)


# ---------- Helpers HTML (Bodega / Alcohol / Alérgenos) ----------

def extract_bodega_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")
    for tr in soup.select('tr[data-testid="wineFactRow"]'):
        header_span = tr.find("span", string=lambda t: t and "Bodega" in t)
        if header_span:
            td = tr.find('td')
            if td: return td.get_text(strip=True)
    return None

def extract_alcohol_allergens_from_html(html: str):
    soup = BeautifulSoup(html, "html.parser")
    alcohol_val = None
    allergens_set = set()

    for tr in soup.select('tr[data-testid="wineFactRow"]'):
        label_span = tr.find("span", class_=lambda c: c and "wineFacts__headerLabel" in c)
        if not label_span: continue
        label = label_span.get_text(strip=True)
        td = tr.find("td")
        if not td: continue
        value_text = td.get_text(strip=True)

        if any(x in label for x in ["Contenido de alcohol", "Alcohol content"]):
            m = re.search(r"([\d.,]+)", value_text)
            if m: alcohol_val = m.group(1).replace(",", ".")

        if any(x in label for x in ["Alérgenos", "Allergens"]):
            vlow = value_text.lower()
            if "sulfit" in vlow: allergens_set.add("sulfitos")
            if "leche" in vlow or "milk" in vlow: allergens_set.add("lácteos")
            if "huevo" in vlow or "egg" in vlow: allergens_set.add("huevo")

    return {"alcohol_html": alcohol_val, "alergenos_html": ", ".join(sorted(allergens_set)) if allergens_set else None}


# ---------- Extracción principal mejorada ----------

def extract_wine_from_html(html: str, source_file: str):
    # Inicializamos todos los campos en None
    res = {k: None for k in ["wine_id", "nombre", "seo", "precio", "rating", "año", "n_ratings", "bodega", "uvas", "region", "wine_style", "alcohol", "alergenos", "descripcion"]}
    res["source_file"] = source_file

    try:
        wpi = extract_wpi_json(html)
    except Exception as e:
        # Si falla el JSON principal, intentamos lo mínimo por HTML o devolvemos vacío
        res["bodega"] = extract_bodega_from_html(html)
        return res

    wine = wpi.get("wine") or {}
    vintage = wpi.get("vintage") or {}
    
    # 1. DETERMINAR EL AÑO (Clave para filtrar el precio)
    # Prioridad: Campo vintage_year -> año en el nombre del vino
    year = wpi.get("vintage_year") or vintage.get("year")
    nombre_vino = wine.get("name") or vintage.get("name") or ""
    
    if not year or str(year) == "1946":
        match_year = re.search(r"(19|20)\d{2}", nombre_vino)
        if match_year:
            year = int(match_year.group())
    res["año"] = year

    # 2. IDENTIDAD DEL VINO
    res["wine_id"] = vintage.get("id") or wine.get("id")
    res["nombre"] = nombre_vino
    res["bodega"] = (wine.get("winery") or {}).get("name") or extract_bodega_from_html(html)
    res["region"] = (wine.get("region") or {}).get("name")
    res["seo"] = wine.get("seo_name")
    res["descripcion"] = wine.get("description")
    res["wine_style"] = (wine.get("wine_style") or {}).get("name")

    # 3. EXTRAER EL PRECIO CORRECTO (El que coincide con el año)
    ppa = wpi.get("prices_and_availabilities") or wpi.get("prices_and_availability") or []
    if isinstance(ppa, dict): ppa = [ppa]

    precio_encontrado = None
    for entry in ppa:
        avail = entry.get("availability") or {}
        vint_info = avail.get("vintage") or {}
        year_in_ppa = vint_info.get("year")
        
        # Filtro: Solo si el año de la oferta coincide con el año del vino analizado
        if str(year_in_ppa) == str(res["año"]) or res["año"] is None:
            price_data = avail.get("price") or avail.get("median") or {}
            if price_data.get("amount") is not None:
                precio_encontrado = float(price_data.get("amount"))
                break # Encontrado el precio de la añada correcta

    # Si no hubo coincidencia exacta de año, intentamos pillar el primer precio de la lista como fallback
    if precio_encontrado is None and ppa:
        first_avail = ppa[0].get("availability") or {}
        price_data = first_avail.get("price") or first_avail.get("median") or {}
        if price_data.get("amount"):
            precio_encontrado = float(price_data.get("amount"))

    res["precio"] = precio_encontrado

    # 4. RATINGS Y ESTADÍSTICAS
    stats = vintage.get("statistics") or wine.get("statistics") or {}
    res["rating"] = stats.get("ratings_average")
    res["n_ratings"] = stats.get("ratings_count")

    # 5. UVAS
    grapes = wine.get("grapes") or []
    res["uvas"] = ", ".join([g.get("name") for g in grapes if g.get("name")])

    # 6. ALCOHOL Y ALÉRGENOS (JSON + HTML Fallback)
    facts_raw = wpi.get("wine_facts_raw") or {}
    res["alcohol"] = facts_raw.get("alcohol")
    
    aler_list = []
    if facts_raw.get("contains_sulfites_allergens"): aler_list.append("sulfitos")
    if facts_raw.get("contains_milk_allergens"): aler_list.append("lácteos")
    res["alergenos"] = ", ".join(aler_list) if aler_list else None

    # Si el JSON no traía estos datos técnicos, miramos en las tablas HTML
    if not res["alcohol"] or not res["alergenos"]:
        h_facts = extract_alcohol_allergens_from_html(html)
        if not res["alcohol"]: res["alcohol"] = h_facts["alcohol_html"]
        if not res["alergenos"]: res["alergenos"] = h_facts["alergenos_html"]

    return res

# ---------- Bucle de ejecución sobre la carpeta ----------

def main():
    folder = "wine_html_1" # Asegúrate de que este es el nombre de tu carpeta
    files = sorted(glob.glob(os.path.join(folder, "*.html")))

    if not files:
        print(f"No se encontraron archivos en {folder}")
        return

    print(f"Procesando {len(files)} archivos...")
    results = []

    for i, path in enumerate(files, 1):
        with open(path, encoding="utf-8") as f:
            html = f.read()
        
        row = extract_wine_from_html(html, os.path.basename(path))
        results.append(row)
        
        if i % 50 == 0:
            print(f" -> {i} archivos procesados...")

    # Exportar a Excel
    df = pd.DataFrame(results)
    output = "prueba1.xlsx"
    df.to_excel(output, index=False)
    print(f"¡Éxito! Datos guardados en {output}")

if __name__ == "__main__":
    main()

Procesando 296 archivos...
 -> 50 archivos procesados...
 -> 100 archivos procesados...
 -> 150 archivos procesados...
 -> 200 archivos procesados...
 -> 250 archivos procesados...
¡Éxito! Datos guardados en prueba1.xlsx


In [5]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

# --- CONFIGURACIÓN ---
CARPETA_HTML = 'wine_html_1'
ARCHIVO_SALIDA = 'prueba 2_v8_final.xlsx'

def limpiar_texto(texto):
    if texto:
        return texto.strip().replace('\n', ' ').replace('\r', '').replace('  ', ' ')
    return ""

def extraer_precio_con_botones(soup_elemento):
    if not soup_elemento: return "N/D"
    # 1. Clases estándar
    precio_lista = soup_elemento.find('span', class_=re.compile(r'vintageListRow__vivinoPrice', re.IGNORECASE))
    if precio_lista: return limpiar_texto(precio_lista.get_text())
    # 2. Botones
    botones = soup_elemento.find_all(['a', 'button'])
    for btn in botones:
        texto_btn = btn.get_text()
        if '€' in texto_btn:
            match = re.search(r'(\d+[,.]\d{2})\s*€', texto_btn)
            if match: return match.group(0)
    # 3. Fallback
    precio_span = soup_elemento.find('span', class_=re.compile(r'price|Value|amount', re.IGNORECASE))
    if precio_span: return limpiar_texto(precio_span.get_text())
    precio_texto = soup_elemento.find(string=re.compile(r'\d+[,.]\d+\s*€'))
    if precio_texto: return limpiar_texto(precio_texto)
    return "N/D"

def extraer_alcohol_global(soup):
    try:
        texto_pagina = soup.get_text()
        match_explicito = re.search(r'Alcohol\D{0,10}(\d{1,2}[.,]?\d{0,1})\s*%', texto_pagina, re.IGNORECASE)
        if match_explicito: return match_explicito.group(1) + "%"
        candidatos = re.findall(r'\b(1[0-6]|[9])([.,]\d)?\s*%', texto_pagina)
        for c in candidatos:
            valor = "".join(c)
            return f"{valor}%"
    except: pass
    return "N/D"

def extraer_datos_header_robusto(soup):
    """
    Intenta sacar el Rating y N_Ratings de la parte superior de la página
    usando múltiples estrategias para cubrir distintos diseños de Vivino.
    """
    rating = "-"
    n_ratings = "-"

    try:
        # --- 1. RATING ---
        # Estrategia A: Clases conocidas
        r_div = soup.find('div', class_=re.compile(r'vivinoRating_averageValue|average_number|indicator_average', re.IGNORECASE))
        if r_div:
            rating = limpiar_texto(r_div.get_text())
        
        # Estrategia B: Buscar número decimal grande aislado (ej: "4.2")
        if rating == "-" or not re.match(r'^\d[.,]\d$', rating):
            # Buscando divs que contengan SOLO un número decimal (común en header)
            candidatos = soup.find_all('div', string=re.compile(r'^[3-5][.,]\d$'))
            if candidatos:
                rating = candidatos[0].get_text()

        # --- 2. N_RATINGS ---
        # Estrategia A: Clases conocidas
        nr_div = soup.find('div', class_=re.compile(r'vivinoRating_caption|vivinoRating_count|indicator__count|reviewCount', re.IGNORECASE))
        if nr_div:
            texto_nr = nr_div.get_text()
            n_ratings = re.sub(r'\D', '', texto_nr) # Deja solo numeros
        
        # Estrategia B: Buscar texto "ratings" o "valoraciones"
        if n_ratings == "-" or n_ratings == "":
            # Buscamos en enlaces o spans texto tipo "1188 ratings"
            elem_texto = soup.find(string=re.compile(r'(\d+[.,]?\d*)\s*(ratings|valoraciones|reviews)', re.IGNORECASE))
            if elem_texto:
                # Extraer solo los digitos
                n_ratings = re.sub(r'\D', '', elem_texto)

    except Exception as e:
        pass

    return rating, n_ratings

def procesar_archivo(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        
        soup = BeautifulSoup(content, 'html.parser')
        datos_archivo = []

        # --- DATOS GENERALES ---
        h1 = soup.find('h1')
        nombre_vino = limpiar_texto(h1.get_text()) if h1 else "Desconocido"
        
        bodega_elem = soup.find('a', attrs={'data-testid': 'winery-name-link'})
        if not bodega_elem: bodega_elem = soup.find('a', href=re.compile(r'/wineries/'))
        bodega = limpiar_texto(bodega_elem.get_text()) if bodega_elem else "Desconocida"

        region_elem = soup.find('a', attrs={'data-testid': 'vintage-page-country'})
        if not region_elem: region_elem = soup.find('a', href=re.compile(r'/wine-regions/'))
        region = limpiar_texto(region_elem.get_text()) if region_elem else "Desconocida"

        link_canonico = soup.find('link', rel='canonical')
        wine_id = "N/D"
        if link_canonico and 'href' in link_canonico.attrs:
            match_id = re.search(r'/w/(\d+)', link_canonico['href'])
            if match_id: wine_id = match_id.group(1)

        meta_desc = soup.find('meta', attrs={'name': 'description'})
        descripcion = ""
        uvas = "N/D"
        if meta_desc and 'content' in meta_desc.attrs:
            contenido_desc = meta_desc['content']
            descripcion = limpiar_texto(contenido_desc)
            match_uvas = re.search(r'Hecho con (.+?)\.', contenido_desc)
            if match_uvas: uvas = match_uvas.group(1)

        alcohol = extraer_alcohol_global(soup)
        
        alergenos = []
        if soup.find(string=re.compile(r'Sulfitos', re.IGNORECASE)): alergenos.append("Sulfitos")
        str_alergenos = ", ".join(alergenos) if alergenos else "N/D"

        # --- EXTRACCIÓN HEADER (Datos de respaldo) ---
        rating_header, resenas_header = extraer_datos_header_robusto(soup)
        
        # Año Principal (del título)
        match_ano = re.search(r'\b(19|20)\d{2}\b', nombre_vino)
        ano_principal = match_ano.group(0) if match_ano else "N.V."

        datos_base = {
            'Wine ID': wine_id,
            'Nombre': nombre_vino,
            'Bodega': bodega,
            'Región': region,
            'Uvas': uvas,
            'Alcohol': alcohol,
            'Alergenos': str_alergenos,
            'Descripción': descripcion,
            'Archivo': os.path.basename(filepath)
        }

        # --- 2. PROCESAR AÑADAS ---
        filas_anadas = soup.find_all('div', class_=re.compile(r'vintageListRow__vintageListRow', re.IGNORECASE))
        
        if filas_anadas:
            for fila in filas_anadas:
                fila_datos = datos_base.copy()
                
                # AÑO
                ano_elem = fila.find('span', class_=re.compile(r'vintageListRow__year', re.IGNORECASE))
                ano = limpiar_texto(ano_elem.get_text()) if ano_elem else "N.V."
                if ano == "N.V.":
                    match = re.search(r'\b(19|20)\d{2}\b', fila.get_text())
                    if match: ano = match.group(0)
                
                # RATING & RESEÑAS (Fila específica)
                rating_div = fila.find('div', class_=re.compile(r'vivinoRating_averageValue|average__number', re.IGNORECASE))
                rating = limpiar_texto(rating_div.get_text()) if rating_div else "-"
                
                resenas_div = fila.find('div', class_=re.compile(r'vivinoRating_caption|vivinoRating_count', re.IGNORECASE))
                resenas = limpiar_texto(resenas_div.get_text()) if resenas_div else "-"
                if resenas != "-": resenas = re.sub(r'\D', '', resenas)

                # --- CORRECCIÓN: Si la fila está vacía y es el año principal, usar Header ---
                if (rating == "-" or rating == "") and ano == ano_principal:
                    rating = rating_header
                
                if (resenas == "-" or resenas == "") and ano == ano_principal:
                    resenas = resenas_header

                # PRECIO
                precio = extraer_precio_con_botones(fila)

                fila_datos['Año'] = ano
                fila_datos['Rating'] = rating
                fila_datos['N_Ratings'] = resenas
                fila_datos['Precio'] = precio
                
                datos_archivo.append(fila_datos)

        # --- 3. SI NO HAY LISTA ---
        if not datos_archivo:
            fila_unica = datos_base.copy()
            fila_unica['Año'] = ano_principal
            fila_unica['Rating'] = rating_header
            fila_unica['N_Ratings'] = resenas_header
            
            # Precio ficha única
            precio_container = soup.find('span', class_=re.compile(r'purchaseAvailability__currentPrice|price'))
            if precio_container:
                fila_unica['Precio'] = extraer_precio_con_botones(precio_container)
            else:
                 # Fallback general
                fila_unica['Precio'] = extraer_precio_con_botones(soup.body)

            datos_archivo.append(fila_unica)

        return datos_archivo

    except Exception as e:
        print(f"Error en {filepath}: {e}")
        return []

# --- MAIN ---
def main():
    if not os.path.exists(CARPETA_HTML):
        print(f"❌ Carpeta '{CARPETA_HTML}' no encontrada.")
        return

    archivos = [f for f in os.listdir(CARPETA_HTML) if f.endswith('.html')]
    print(f"📂 Procesando {len(archivos)} archivos...")

    todos = []
    for archivo in archivos:
        datos = procesar_archivo(os.path.join(CARPETA_HTML, archivo))
        todos.extend(datos)

    if todos:
        df = pd.DataFrame(todos)
        cols = ['Wine ID', 'Nombre', 'Bodega', 'Región', 'Año', 'Rating', 'N_Ratings', 'Precio', 'Alcohol', 'Uvas', 'Alergenos', 'Descripción', 'Archivo']
        df = df[[c for c in cols if c in df.columns]]
        
        df.to_excel(ARCHIVO_SALIDA, index=False)
        print(f"✅ ¡Hecho! Datos guardados en: {ARCHIVO_SALIDA}")
        # Verificación para el vino problemático
        check = df[df['Archivo'] == '1169745.html']
        if not check.empty:
            print("--- Verificación Balbas Ritus ---")
            print(check[['Año', 'Rating', 'N_Ratings']].head())
    else:
        print("⚠️ No se encontraron datos.")

if __name__ == "__main__":
    main()

📂 Procesando 296 archivos...
✅ ¡Hecho! Datos guardados en: prueba 2_v8_final.xlsx
--- Verificación Balbas Ritus ---
      Año Rating N_Ratings
235  2017    4,4       149
236  2019    4,3       212
237  2018    4,3       142
238  2020    4,4        39
239  2016    4,3        90
